In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import os
import pylcs
from typing import Union, List, Optional
from pprint import pprint

import transformers
from transformers import (AutoTokenizer, AutoModelForTokenClassification,
                         pipeline, TrainingArguments, Trainer,
                         DataCollatorForTokenClassification, EarlyStoppingCallback)

import datasets
from datasets import Dataset, Features, Sequence, Value, ClassLabel, load_dataset, load_from_disk

import torch
import evaluate
import re


print(transformers.__version__)

import os
os.environ['WANDB_DISABLED'] = 'true'

task = "ner" # Should be one of "ner", "pos" or "chunk"
# model_checkpoint = "distilbert-base-uncased"
batch_size = 16
seqeval = evaluate.load("seqeval")

4.51.3


In [5]:
path_to_model = "/home/mikhail/Documents/pandan_study/vkr/vulns_scanner/mikhail_code/models/nuner2_v1_150325/best_model_tmp"
final_tokenizer = AutoTokenizer.from_pretrained(path_to_model, use_fast=True, add_prefix_space=True, local_files_only=True)
final_model = AutoModelForTokenClassification.from_pretrained(path_to_model, local_files_only=True)


In [6]:
s = 'Improper Restriction of XML External Entity Reference in GitHub repository hazelcast/hazelcast in 5.1-BETA-1'

token_classifier = pipeline(
    "token-classification", model=final_model, aggregation_strategy="first", tokenizer=final_tokenizer
)
res = token_classifier(s)
for i, r in enumerate(res):
    # print('Entity: '+ r['entity_group'] + '   Word: ' + r['word'])
    print('Entity: '+ r['entity_group'] + '   Word: ' + r['word'] + '   Prob: ' + str(r['score']))

Device set to use cpu


Entity: version   Word:  5.1-BETA-1   Prob: 0.99713945


/home/mikhail/.local/lib/python3.10/site-packages/transformers/pipelines/token_classification.py:398: UserWarning: Tokenizer does not support real words, using fallback heuristic
  warnings.warn(


In [ ]:
def run(input_string: str) -> str:
    token_classifier = pipeline(
        "token-classification", model=final_model, aggregation_strategy="first", tokenizer=final_tokenizer
    )
    res = token_classifier(input_string)
    output = ''
    for i, r in enumerate(res):
        # print('Entity: '+ r['entity_group'] + '   Word: ' + r['word'])
        output += str(f'{i+1}. ' + 'Entity: ' + r['entity_group'] + '   Word: ' + r['word'] + '   Prob: ' + str(r['score']) + '\n')
    # print(output)
    if output == '':
        output = 'No NER found'
    return output

In [8]:
run('Out-of-Bounds Read in ip_set_bitmap_ip.c in Google ChromeOS Kernel Versions 6.1, 5.15, 5.10, 5.4, 4.19. on All devices where Termina is used allows an attacker with CAP_NET_ADMIN privileges to cause memory corruption and potentially escalate privileges via crafted ipset commands.')

Device set to use cpu
/home/mikhail/.local/lib/python3.10/site-packages/transformers/pipelines/token_classification.py:398: UserWarning: Tokenizer does not support real words, using fallback heuristic
  warnings.warn(


'1. Entity: vendor   Word:  Google   Prob: 0.9997322\n2. Entity: version   Word:  6.1,   Prob: 0.99980503\n3. Entity: version   Word:  5.15,   Prob: 0.9998227\n4. Entity: version   Word:  5.10,   Prob: 0.99973506\n5. Entity: version   Word:  5.4,   Prob: 0.9997838\n6. Entity: version   Word:  4.19.   Prob: 0.999775\n'